# Text Encoding

Trong quá trình xử lý dữ liệu thực tế, rất nhiều thuộc tính (cột) xuất hiện dưới dạng văn bản như tên danh mục, trạng thái, mô tả, bình luận,… Tuy nhiên, các mô hình Machine Learning không thể làm việc trực tiếp với dữ liệu dạng text. Vì vậy, bước mã hóa dữ liệu (Text Encoding) là một phần quan trọng trong giai đoạn tiền xử lý dữ liệu (data preprocessing).

Notebook này trình bày các kỹ thuật phổ biến để chuyển đổi văn bản thành dạng số, giúp mô hình có thể hiểu và học từ dữ liệu:

1. One-Hot Encoding – biến mỗi giá trị phân loại thành một vector nhị phân
2. Target Encoding – mã hoá theo mục tiêu

# 0. Nguồn dữ liệu

In [1]:
import pandas as pd

df = pd.read_csv('../data/interim/combined_interim.csv')

fields = ['origin', 'status', 'brand', 'model', 'fuel_type', 'style']

# for col in fields:
#     print("\n" + "="*50)
#     print(f"THỐNG KÊ FIELD: {col.upper()}")
#     print("="*50)
#
#     vc = df[col].value_counts(dropna=False)
#
#     print(f"➡ Số giá trị unique: {df[col].nunique(dropna=False)}")
#
#     print(vc.to_string())
#     print("="*50 + "\n")


# 1. One-hot field
Một số trường dữ liệu có loại dữ liệu thuộc một tập hữu hạn trong khoảng dưới 30 loại (có thể chọn đến 50) sẽ được encoding theo one-hot

In [2]:
import unidecode  # để chuyển dấu tiếng Việt sang ASCII

# ======= Chuẩn hóa và gom nhãn cho 'status'========
status_mapping = {
    'đã sử dụng': 'used',
    'xe đã dùng': 'used',
    'xe mới': 'new',
    'mới': 'new'
}

df['status'] = (
    df['status']
    .astype(str)          # chuyển thành chuỗi
    .str.lower()          # lowercase để map dễ dàng
    .str.strip()          # loại bỏ khoảng trắng đầu/cuối
    .replace(status_mapping)  # gom nhãn
)
print(df['status'].value_counts())

# ========= Chuẩn hóa và gom nhãn cho 'fuel_type' =========
fuel_mapping = {
    'xăng': 'gasoline',
    'dầu': 'diesel',
    'điện': 'electric',
    'hybrid': 'hybrid'
}

df['fuel_type'] = (
    df['fuel_type']
    .astype(str)
    .str.lower()
    .str.strip()
    .replace(fuel_mapping)
)

print(df['fuel_type'].value_counts())

# ========= Chuẩn hóa và gom nhãn cho 'fuel_type' =========
# Lấy danh sách các giá trị origin
origins = df['origin'].unique()

# Tạo mapping: chuẩn hóa tên
origin_mapping = {orig: unidecode.unidecode(orig).lower().replace(' ', '_') for orig in origins}

# Áp dụng mapping
df['origin'] = df['origin'].map(origin_mapping)

print(df['origin'].value_counts())

# ========= Chuẩn hóa và gom nhãn cho 'brand' =========
total_samples = len(df)
threshold = 0.005 * total_samples  # 0.5% tổng mẫu

# Đếm tần suất từng brand
brand_counts = df['brand'].value_counts()

# Gom các brand ít xuất hiện thành 'other'
df['brand'] = df['brand'].apply(lambda x: x if brand_counts[x] >= threshold else 'other')

# ========= Chuẩn hóa và gom nhãn cho 'style' =========
style_mapping = {
    # SUV
    'SUV / Cross over': 'SUV',
    'SUV': 'SUV',

    # Sedan
    'Sedan': 'Sedan',

    # Hatchback
    'Hatchback': 'Hatchback',

    # Minivan / Van
    'Minivan (MPV)': 'Minivan',
    'Van/Minivan': 'Minivan',
    'Van': 'Minivan',

    # Pickup / Bán tải
    'Bán tải / Pickup': 'Pickup',
    'Pick-up (bán tải)': 'Pickup',

    # Coupe
    'Coupe': 'Coupe',
    'Coupe (2 cửa)': 'Coupe',

    # Convertible
    'Convertible/Cabriolet': 'Convertible',

    # Kiểu dáng khác / unknown
    'Kiểu dáng khác': 'other',
    None: 'unknown',  # xử lý NaN
    'NaN': 'unknown'
}

# Áp dụng mapping
df['style'] = df['style'].map(style_mapping).fillna(df['style'])
# Chuẩn hóa tên: lowercase + no dấu + replace space bằng _
df['style'] = df['style'].apply(lambda x: unidecode.unidecode(str(x)).lower().replace(' ', '_'))

# Kiểm tra kết quả
print(df['style'].value_counts())

status
used    1781
new      219
Name: count, dtype: int64
fuel_type
gasoline    1583
diesel       252
electric     106
hybrid        59
Name: count, dtype: int64
origin
nhap_khau             504
lap_rap_trong_nuoc    496
viet_nam              424
dang_cap_nhat         225
han_quoc               76
thai_lan               74
nhat_ban               60
nuoc_khac              51
my                     33
duc                    30
trung_quoc             13
an_do                  10
dai_loan                4
Name: count, dtype: int64
style
suv            752
sedan          490
nan            203
minivan        199
hatchback      172
crossover       89
pickup          73
other           12
coupe            8
convertible      2
Name: count, dtype: int64


# 2. Target Encoding

# 3. Lưu thay đổi

In [3]:
# === One-Hot Encoding ===
df = pd.get_dummies(df,
                    columns=['status', 'fuel_type', 'origin', 'style', 'brand'],
                    prefix=['status', 'fuel', 'origin', 'style', 'brand'])

# Chuyển True/False sang 0/1 cho các cột one-hot
dummy_cols = [col for col in df.columns if col.startswith('status_')
                                      or col.startswith('fuel_')
                                      or col.startswith('origin_')
                                      or col.startswith('style_')
                                      or col.startswith('brand_')]
df[dummy_cols] = df[dummy_cols].astype(int)

# === Giữ nguyên kiểu dữ liệu các cột số ===
numeric_cols = ['price', 'odometer', 'seats', 'year']

for col in numeric_cols:
    if col in df.columns:
        # price luôn int64
        if col == 'price':
            df[col] = df[col].astype('int64')
        # year: Int64 nếu có NaN, int64 nếu không NaN
        elif col == 'year':
            df[col] = df[col].astype('Int64')
        # odometer: Int64 nếu có NaN, int64 nếu không
        elif col == 'odometer':
            if df[col].notna().all():
                df[col] = df[col].astype('int64')
            else:
                df[col] = df[col].astype('Int64')
        # seats: Int64 nếu có NaN, int64 nếu không
        elif col == 'seats':
            if df[col].notna().all():
                df[col] = df[col].astype('int64')
            else:
                df[col] = df[col].astype('Int64')

# === Xuất CSV ===
csv_file = '../data/interim/encoding_interim.csv'
df.to_csv(csv_file, index=False, encoding='utf-8-sig')

# === Kết quả ===
print("\n✓ Đã áp dụng One-Hot Encoding")
print(f"Số cột sau One-Hot Encoding: {df.shape[1]}")
print(f"\nCác cột mới:")
print([col for col in df.columns if col.startswith('status_') or col.startswith('fuel_')])
print("\nDữ liệu sau encoding:")
print(df.head())



✓ Đã áp dụng One-Hot Encoding
Số cột sau One-Hot Encoding: 58

Các cột mới:
['status_new', 'status_used', 'fuel_diesel', 'fuel_electric', 'fuel_gasoline', 'fuel_hybrid']

Dữ liệu sau encoding:
       price  odometer    model  year  seats  status_new  status_used  \
0  149000000    159000    Yaris  2007   <NA>           0            1   
1  360000000     99000        6  2016      5           0            1   
2  195000000         1    Hiace  2008      6           0            1   
3  416000000     92000    Civic  2019   <NA>           0            1   
4  250000000     15000  Orlando  2017      7           0            1   

   fuel_diesel  fuel_electric  fuel_gasoline  ...  brand_Mitsubishi  \
0            0              0              1  ...                 0   
1            0              0              1  ...                 0   
2            1              0              0  ...                 0   
3            0              0              1  ...                 0   
4           